In [2]:
import sys
sys.path.append('../src')
from ase.io import read
vasp=read('optimized.vasp',format='vasp')
natom=len(vasp.positions)
cell=vasp.cell
positions=vasp.positions
masses=vasp.get_masses()

In [3]:
import numpy as np
nmodes=natom*3
lammps_dyn=np.loadtxt('Dyn.form').reshape((nmodes,nmodes))

In [4]:
structure_file='optimized.vasp'
from thermal_conductivity_AF import *
Vx,Vy,Vz=get_Vij_from_flat(structure_file,lammps_dyn)

In [5]:
#check part
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vx-GULP_Vijx) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [6]:
GULP_Vijy=np.loadtxt('Vijy.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vy-GULP_Vijy) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [7]:
GULP_Vijz=np.loadtxt('Vijz.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vz-GULP_Vijz) > 0.01)

(array([], dtype=int64), array([], dtype=int64))

In [8]:
#scale to convert THz or cm-1 unit
#Boltzmanns constant (J/K)
boltz = 1.38066244e-23

#Planck's constant (Js)
planck = 6.62617636e-34

#Speed of light (in cm/s)
speedl = 2.99792458e10

#eV -> Joules

evtoj = 1.6021917e-19

#Avogadros number
avogadro = 6.022045e23

#THz:1e-12
#AngAngkgTHzTHz
#scale factor to eigenvalue
scale_THz =np.sqrt(0.1*evtoj*avogadro)/(2.0*np.pi)
scale_cm =np.sqrt(1.0e23*evtoj*avogadro)/speedl/(2.0*np.pi)

In [9]:
scale_cm, scale_THz, scale_cm/scale_THz

(521.4692040170344, 15.63325344435702, 33.35640951981521)

In [18]:
#using eigh(Hermite) much faster & order of eigenvalue is similar to that in GULP
eigenvalue, eigenvector=np.linalg.eigh(lammps_dyn)

In [19]:
freq=[]
with open("freq_LAMMPS.dat",'w') as fL:
    for i in range(nmodes):
        if eigenvalue[i] <0.0:
            val=np.sqrt(-eigenvalue[i])*scale_cm
            freq.append(val)
            fL.write(str(val)+'\n')
        else:
            val=np.sqrt(eigenvalue[i])*scale_cm
            fL.write(str(val)+'\n')
            freq.append(val)


In [38]:
GULP_eig=np.loadtxt('eigr.dat').reshape(nmodes,nmodes)

In [39]:
GULP_eig[:,3]

array([ 0.00833792, -0.02958731, -0.01382491, ...,  0.00113291,
       -0.00667961, -0.00602454])

In [40]:
eigenvector[:,3]

array([ 0.00834461, -0.02960095, -0.01383089, ...,  0.00113445,
       -0.00668173, -0.0060268 ])

In [52]:
np.argmax(np.abs(eigenvector-GULP_eig))

660478

In [33]:
freq[0:3]=[ -1.917099908336566E-005,1.544513285771906E-005,2.897206441232811E-005]

In [34]:
Sx,Sy,Sz=get_Sij(Vx,Vy,Vz,eigenvector,freq)

In [21]:
GULP_Sijx=np.loadtxt('Sijx.dat').reshape((nmodes,nmodes))
GULP_Sijy=np.loadtxt('Sijy.dat').reshape((nmodes,nmodes))
GULP_Sijz=np.loadtxt('Sijz.dat').reshape((nmodes,nmodes))


(array([   0,    0,    0, ..., 1535, 1535, 1535]),
 array([   4,    5,    6, ..., 1513, 1516, 1523]))

In [35]:
np.where(np.abs(Sx-GULP_Sijx) > 0.01)

(array([   1,    1,    1, ..., 1535, 1535, 1535]),
 array([   4,    5,    6, ..., 1513, 1516, 1523]))

In [36]:
GULP_Sijx[1,0:6]

array([ 0.00000000e+00,  1.21430643e-17, -1.45698759e-16,  3.40375494e+00,
        3.35875123e+00,  2.74822320e+00])

In [37]:
Sx[1,0:6]

array([ 0.00000000e+00, -1.62793749e-10, -1.83455468e-09,  3.41131667e+00,
        1.38049350e+00, -3.06387826e+00])

In [42]:
omega=freq
nfreqmin=0
inv_omega=np.zeros(nmodes)
for i in range(nmodes):
    if (nfreqmin==0 and omega[i]> 0.01):
        nfreqmin = i
    if omega[i] >0.0:
        inv_omega[i]=1.0/np.sqrt(omega[i])
    else:
        inv_omega[i]=0.0

In [43]:
tmpx=np.dot(Vx,eigenvector)
EVijx=np.dot(eigenvector.T,tmpx)

In [44]:
Sijx=np.zeros((nmodes,nmodes))
for i in range(nmodes):
    for j in range(nmodes):
        Sijx[i,j]=EVijx[i,j]*(omega[i]+omega[j])*inv_omega[i]*inv_omega[j]

In [45]:
Sijx[1,0:6]

array([ 0.00000000e+00, -1.62793749e-10, -1.83455468e-09,  3.41131667e+00,
        1.38049350e+00, -3.06387826e+00])

In [53]:
constant = ((1.0e-17*evtoj*avogadro)**0.5)*(scale_cm**3)
constant = np.pi*constant/48.0

In [54]:
#get mean of frequency spacing
dwavg=0.0
for i in range(len(omega)-1):
    if omega[i] > 0.0:
        dwavg+=omega[i+1]-omega[i]
    elif omega[i+1] >0.0:
        dwavg+=omega[i+1]

dwavg=dwavg/(len(omega)-1)
dwavg

0.4377526769013716

In [55]:
Di=np.zeros(len(omega))
nmodes=len(omega)
bfactor=5.0
broad = bfactor*dwavg
for i in range(nmodes):
    Di_loc = 0.0
    for j in range(nmodes):
        if(omega[i] > 0.01):
            dwij = (1.0/np.pi)*broad/( (omega[j] - omega[i])**2 + broad**2 )
            if(dwij > 0.01):
                Di_loc = Di_loc + dwij*Sx[j,i]**2+dwij*Sy[j,i]**2+dwij*Sz[j,i]**2
    Di[i] = Di[i] + Di_loc*constant/(omega[i]**2)

In [56]:
Di

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       9.34287666e-10, 3.85287912e-29, 1.63156491e-23])

In [63]:
from ase.io import read
atoms=read(structure_file,format='vasp')
vol = atoms.get_volume()
kappafct = 1.0e30/vol
temperature=300
cmfact = planck*speedl/(boltz*temperature)
kappa_info=[]

with open('kappa_out','w') as kf:
    kf.write('frequency[cm-1]   Diffusivity[cm^2/s]   Thermal_conductivity[W/mK] \n')
    for i in range(nmodes):
        xfreq = omega[i]*cmfact
        expfreq = np.exp(xfreq)
        cv_i = boltz*xfreq*xfreq*expfreq/(expfreq - 1.0)**2
        kappa_info.append([omega[i],Di[i]*1.0e4,cv_i*kappafct*Di[i]])
        kf.write('{0:8f}  {1:12f}  {2:12f}\n'.format(omega[i],Di[i]*1.0e4,cv_i*kappafct*Di[i]))